In [1]:
import sys
from pathlib import Path
# Add src directory to path to import organoid_analysis package
sys.path.insert(0, str(Path('..') / 'src'))
import os
import pandas as pd
import numpy as np
from cellpose import models, core, io
import pyclesperanto_prototype as cle 
from skimage.measure import regionprops_table
from tifffile import imwrite, imread
from organoid_analysis.utils import list_images, read_image, extract_scaling_metadata, remap_labels, segment_organoids_from_cp_labels, extract_organoid_stats_and_merge

io.logger_setup() # run this to get printing of progress

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose models
model = models.CellposeModel(gpu=True, model_type="cyto3") 

creating new log file
2026-02-18 04:51:02,212 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2026-02-18 04:51:02,212 [INFO] 
cellpose version: 	3.1.1.3 
platform:       	win32 
python version: 	3.10.19 
torch version:  	2.5.0
2026-02-18 04:51:02,267 [INFO] ** TORCH CUDA version installed and working. **
2026-02-18 04:51:02,268 [INFO] >> cyto3 << model set to be used
2026-02-18 04:51:02,269 [INFO] ** TORCH CUDA version installed and working. **
2026-02-18 04:51:02,269 [INFO] >>>> using GPU (CUDA)
2026-02-18 04:51:02,303 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cyto3
2026-02-18 04:51:02,363 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [2]:
# Define the markers you wish to analyze and its cellular compartment (i.e. cell, membrane or nuclei)
# ("channel_name", position, location)
markers = [("Occludin_RFP", 0, "membrane"), ("Claudin_FITC", 1, "membrane"), ("Occludin_RFP", 0, "cell"), ("DAPI", 2, "nuclei")]

In [3]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path(r"\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\20260114_T7_2microns")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\B2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\B6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\C2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\C6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\D2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\D6.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\PhD\\Nikon Spinning Disc\\20260114_T7_2microns\\E2.nd2',
 '\\\\forskning.it.ntnu.no\\ntnu\\mh\\ikom\\cmic_konfokal\\lusie.f.kuraas\\P

In [4]:
folders_to_create = []

# Extract experiment_id from data folder Path object
experiment_id = Path(directory_path).name

# Create a 'results' folder in the root directory
results_folder = Path("results") / experiment_id
folders_to_create.append(results_folder)

# Create cellpose label folders inside the data folder (cell and nuclei)
cellpose_cell_folder = directory_path / "cellpose_cell_labels"
cellpose_nuclei_folder = directory_path / "cellpose_nuclei_labels"
folders_to_create.append(cellpose_cell_folder)
folders_to_create.append(cellpose_nuclei_folder)

for path in folders_to_create:
    try:
        os.makedirs(path)
        print(f"'{path}' folder created successfully.")
    except FileExistsError:
        print(f"'{path}' folder already exists.")

'results\20260114_T7_2microns' folder created successfully.
'\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\20260114_T7_2microns\cellpose_cell_labels' folder already exists.
'\\forskning.it.ntnu.no\ntnu\mh\ikom\cmic_konfokal\lusie.f.kuraas\PhD\Nikon Spinning Disc\20260114_T7_2microns\cellpose_nuclei_labels' folder created successfully.


In [ ]:
# Loop through all .nd2 files in the directory (each file contains multiple xy positions)
for image in images:

    # Read image, apply slicing if needed and return filename and img as a np array
    img, filename = read_image(image, slicing_factor_xy)

    # Extract well_id from filename
    well_id = filename.split("_")[0]

    # Check if results are already present in the results folder and skip the rest of the loop if so
    # Define the per_well_id .csv name
    csv_name = f"{well_id}_per_cell_results.csv"
    
    # Check if there is a previous copy of the .csv file at results_folder / csv_name 
    csv_path = results_folder / csv_name
    # Skip to the next image if the results already exist
    if csv_path.is_file():
        print(f"Skipping {well_id} well analysis: Results already found at: {csv_path}")
        continue  # Skip to the next image if the results already exist

    # Extract x,y,z scaling from .nd2 file metadata in order feed the Z pixel size / XY pixel size ratio into Cellpose
    pixel_size_x, pixel_size_y, voxel_size_z = extract_scaling_metadata(image)

    # Calculate anisotropy ratio:
    z_to_xy_ratio = voxel_size_z / pixel_size_x

    # Empty list to hold per_position final Dataframes
    per_pos_dfs = []

    # Loop through all the positions inside the .nd2 file
    for position in range(img.shape[0]):

        print(f"Analyzing multiposition index {position}")
        # Generate name for Cellpose prediction for current well, position and cellular compartment
        # (suffix, subdir, channels) for each compartment; channels: [cyto_chan, nuclear_chan] for cell; [nuclear_chan, grayscale] for nuclei (indices in cellpose_input 0=mem, 1=nuclei, 2=Cellmask)
        segment_configs = [
            ("cell", "cellpose_cell_labels", [3, 2]),   # Cellmask (2), nuclei (1) in cellpose_input
            ("nuclei", "cellpose_nuclei_labels", [2, 0]),  # nuclei (1), grayscale
        ]

        # Open one of the multipositions in the img file
        single_img = img[position]
        # Input shape for next step has to be C, Z, Y, X
        single_img = single_img.transpose(1, 0, 2, 3)

        # Keep only 3 meaningful input fluorescence channels to mimick Cellpose GUI normalization
        # 0 membrane marker, 2 nuclei, 3 Cellmask (Numpy 0-index) → cellpose_input: 0=mem, 1=nuclei, 2=Cellmask
        cellpose_input = single_img[[0, 2, 3], :, :, :]

        labels = {}
        for suffix, subdir, channels in segment_configs:
            # Construct path to store/check for labels
            cellpose_filename = f"{well_id}_{position}_{suffix}"
            prediction_path = directory_path / subdir / f"{cellpose_filename}.tif"

            # Check if the prediction has already been generated and load it
            if prediction_path.exists():
                labels[suffix] = imread(prediction_path)
            else:
                # Segment cells with Cellpose (corrected for anisotropy and cell diameter)
                labels[suffix], _, _ = model.eval(
                    cellpose_input,
                    channels=channels,
                    diameter=20,
                    do_3D=True,
                    anisotropy=z_to_xy_ratio,
                    normalize=True,
                    flow_threshold=0.4,
                    cellprob_threshold=0.0,
                    min_size=15,
                )
                # Save prediction as .tif to avoid extra Cellpose computations in next rounds
                imwrite(prediction_path, labels[suffix])

        del cellpose_input

        cytoplasm_labels = labels["cell"]
        nuclei_labels = labels["nuclei"]
        # Remap nuclei labels to match their surrounding cytoplasm id
        nuclei_labels = remap_labels(nuclei_labels, cytoplasm_labels)

        # Create a dictionary containing all image descriptors
        # Add multiposition index during BP
        descriptor_dict = {
                    "filename": filename,
                    "well_id": well_id,
                    "multiposition_id": position,
                    }

        props_list = [] # Empty list to hold all per marker feature dataframes

        # Single list of regionprops to request; rename_map is built from this below (all 3D-compatible)
        regionprops_properties = [
            "label",
            "area",                          # number of voxels (volume in voxel units)
            "area_bbox",                     # volume of axis-aligned bounding box
            "area_convex",                   # volume of convex hull of the region
            "area_filled",                   # volume after filling holes
            "axis_major_length",             # length of major axis from inertia tensor (elongation)
            "axis_minor_length",             # length of minor axis (second principal axis in 3D)
            "equivalent_diameter_area",      # diameter of sphere with same volume as region
            "euler_number",                  # topology: objects + holes − tunnels (connectivity)
            "extent",                        # volume / bounding-box volume (fill of the box)
            "feret_diameter_max",            # maximum Feret (caliper) diameter
            "solidity",                      # volume / convex-hull volume (compact vs lobed)
            "inertia_tensor_eigvals",        # eigenvalues of inertia tensor (3 values: shape/orientation)
            "intensity_mean",
            "intensity_min",
            "intensity_max",
            "intensity_std",
        ]

        membrane_labels = None # Variable to check in order to compute membrane_labels just once, avoid repeated GPU ops
        locations_seen = set()  # track locations so we only keep shape columns once per location (avoids duplicate columns on merge)

        # Loop through markers and extract 
        for marker_name, ch_nr, location in markers:
            print(f"Analyzing channel: {marker_name} in {location} ...")

            # Region cell, nuclei or membrane: generate membrane and extract info from that location if needed. Pick label image.
            if location == "cell":
                label_image = cytoplasm_labels
            elif location == "nuclei":
                label_image = nuclei_labels
            elif location == "membrane":
                # Check if membrane_labels have already been precomputed, otherwise generate
                if membrane_labels is None:
                    # Generate membrane by keeping a single pixel edge (might modify later)
                    membrane_labels = cle.reduce_labels_to_label_edges(cytoplasm_labels)
                    membrane_labels = cle.pull(membrane_labels)
                label_image = membrane_labels
            else:
                raise ValueError(f"Unknown location: {location}")

            props = regionprops_table(
                label_image=label_image,
                intensity_image=single_img[ch_nr],
                properties=regionprops_properties,
            )

            # Convert to dataframe
            props_df = pd.DataFrame(props)

            # Rename columns to indicate the specific image; build rename_map from regionprops_properties (all props except label; concentration proxy etc.)
            prefix = f"{location}_{marker_name}"
            rename_map = {"label": "label"}
            for prop in regionprops_properties:
                if prop == "label":
                    continue
                elif prop == "area":
                    rename_map[prop] = f"{location}_area"
                elif prop.startswith("intensity_"):
                    suffix = prop.replace("intensity_", "")
                    rename_map[prop] = f"{prefix}_{suffix}_int"
                else:
                    rename_map[prop] = f"{location}_{prop}"

            props_df.rename(columns=rename_map, inplace=True)

            # Derived columns (use names from rename_map so they stay in sync with regionprops_properties)
            mean_col = rename_map["intensity_mean"]
            max_col = rename_map["intensity_max"]
            area_col = rename_map["area"]
            # Max / mean ratio (puncta vs diffuse signal)
            props_df[f"{prefix}_max_mean_ratio"] = props_df[max_col] / props_df[mean_col].replace(0, np.nan)
            # Total marker content per cell
            props_df[f"{prefix}_sum_int"] = props_df[mean_col] * props_df[area_col]

            # If we already have shape columns for this location (e.g. second membrane marker), keep only label and marker-specific columns to avoid MergeError on duplicate column names
            if location in locations_seen:
                cols_to_keep = ["label"] + [c for c in props_df.columns if c.startswith(prefix + "_")]
                props_df = props_df[cols_to_keep]
            else:
                locations_seen.add(location)

            # Append each props_df to props_list
            props_list.append(props_df)

        # Initialize the df with the first df in the list
        props_df = props_list[0]
        # Start looping from the second df in the list
        for df in props_list[1:]:
            props_df = props_df.merge(df, on="label")

        # Add each key-value pair from descriptor_dict to props_df at the specified position
        insertion_position = 0    
        for key, value in descriptor_dict.items():
            props_df.insert(insertion_position, key, value)
            insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

        # Obtain rough single organoid outlines by fusing Cellpose labels and dilation, merging, erosion morphological operations
        mip_labels, organoid_labels = segment_organoids_from_cp_labels(cytoplasm_labels)

        # Obtain final stats (including per cell and per organoid)
        final_df = extract_organoid_stats_and_merge(mip_labels, organoid_labels, props_df)

        # Append each position results and concatenate later to store on a per well_id(img) basis
        per_pos_dfs.append(final_df)

    # Concatenate all Dataframes in the per_pos_dfs list
    df_well_id = pd.concat(per_pos_dfs, ignore_index=True)

    # Save to CSV
    df_well_id.to_csv(csv_path, index=False)  


Image analyzed: B2
Pixel size: 0.663 µm x 0.663 µm
Voxel (Z-step) size: 2.000 µm
Analyzing multiposition index 0
2026-02-18 04:51:38,086 [INFO] multi-stack tiff read in as having 21 planes 3 channels
2026-02-18 04:51:38,309 [INFO] resizing 3D image with rescale=1.50 and anisotropy=3.017451031446116
2026-02-18 04:51:39,649 [INFO] running YX: 95 planes of size (2040, 2040)
2026-02-18 04:52:16,629 [INFO] 100%|##########| 95/95 [00:36<00:00,  2.57it/s]
2026-02-18 04:52:19,162 [INFO] running ZY: 2040 planes of size (95, 2040)
2026-02-18 04:52:57,337 [INFO] 100%|##########| 2040/2040 [00:38<00:00, 53.44it/s]
2026-02-18 04:52:58,970 [INFO] running ZX: 2040 planes of size (95, 2040)
2026-02-18 04:53:45,193 [INFO] 100%|##########| 2040/2040 [00:46<00:00, 44.15it/s]
2026-02-18 04:53:50,423 [INFO] resizing 3D flows and cellprob to original image size
2026-02-18 04:53:51,751 [INFO] network run in 133.44s
2026-02-18 04:53:53,957 [INFO] masks created in 2.01s
2026-02-18 04:53:56,515 [INFO] multi-st

NotImplementedError: Property perimeter is not implemented for 3D images